In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import progressbar
from sklearn import preprocessing

from pathlib import Path
import ffmpeg
from os.path import isfile, join
from os import listdir

In [2]:
#Define if the target to check the max value need to be postive or not (positive=arrive at last forth of field)
only_positive_targets = False

In [3]:
all_games = pd.read_csv('all_games.csv')
all_games.drop(all_games.columns[[0,1]],1,inplace=True)

all_games_og = pd.read_csv('all_games_other_games.csv')
all_games_og.drop(all_games_og.columns[[0,1]],1,inplace=True)

# all_games_ad = pd.concat([all_games, all_games_og])

In [4]:
partidas = pd.read_csv('all_games_ad_FETW.csv')['partida'].unique()
partidas = partidas[1:2]

In [5]:
df_BPIs_info = all_games_og[(all_games_og['partida'] == partidas[0]) & (all_games_og['interval_frame_size'] >= 500)]
df_BPIs_info.head()

,frame,FETW_end,LAG_start,frame_end,LAG_end,Target_start,interval_frame_size,interval_time,team_possession,target,partida
206,495,662,663,1086.0,918.0,919.0,591.0,17.73,0,0,REDSCAT2
207,1087,1254,1255,1903.0,1735.0,1736.0,816.0,24.48,1,0,REDSCAT2
210,2463,2630,2631,2995.0,2827.0,2828.0,532.0,15.96,0,0,REDSCAT2
212,3058,3225,3226,3591.0,3423.0,3424.0,533.0,15.99,0,1,REDSCAT2
213,3592,3759,3760,4192.0,4024.0,4025.0,600.0,18.00,1,0,REDSCAT2


## Reading the graph metrics

In [6]:
#Path where to find graph metrics csvs
path_grafo_metric = f'grafo_all_bases/{partidas[0]}/'

In [7]:
# Read the graph atribute saved
def read_files_on_fold(mypath):
    """
    Recive the path where the attributes was saved, and return 
    the name of all files in the fold
    """
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return onlyfiles

In [8]:
def read_dataframe_metric(file, mypath):
    """
    Recive the file name and the path of file, return a DataFrame
    with the atributtes of graphs
    """
    metric = pd.read_csv(mypath+'/'+str(file))
    metric.drop(metric.columns[[0]],1, inplace= True)
    metric.fillna(0,inplace= True)
    
    return metric

In [9]:
files = read_files_on_fold(path_grafo_metric)

In [10]:
#List of metrics that we can find the max values
top_metrics = ["locefficiency", "entropy", "eccentricity"]

In [11]:
#Normalizing the values of metrics
min_max_scaler = preprocessing.MinMaxScaler()

In [12]:
def split_underscore(text, idx):
    return int(text.split('_')[idx])

In [13]:
def target_validation(file):
    if only_positive_targets:
        if df_BPIs_info.iloc[split_underscore(file,0)]['target'] == 1:
            return True
        else:
            return False
    else:
        return True

In [14]:
def get_max_values(metric):
    """
    Recives the metric name and return the max values in the metrics
    with the name of the file with the metric.
    """
    list_metrics_t1 = []
    list_metrics_t2 = []
    max_values = {'t1':0,
                  't1_idx':"",
                  't1_norm':0,
                  't2':0,
                  't2_idx':"",
                  't2_norm':0,}
    
    for file in progressbar.progressbar(files):
        if (file.split('_')[1] == metric) & target_validation(file):
            if 't1' in file.split('_')[2]:
                metric_t1 = read_dataframe_metric(file, path_grafo_metric)
                list_metrics_t1.append(metric_t1)
                if metric_t1.max().max() > max_values['t1']:
                    max_values['t1'] = metric_t1.max().max()
                    max_values['t1_idx'] = file
                    max_values['t1_norm'] = min_max_scaler.fit_transform(metric_t1)
            elif 't2' in file.split('_')[2]:
                metric_t2 = read_dataframe_metric(file, path_grafo_metric)
                list_metrics_t2.append(metric_t2)
                if metric_t2.max().max() > max_values['t2']:
                    max_values['t2'] = metric_t1.max().max()
                    max_values['t2_idx'] = file
                    max_values['t2_norm'] = min_max_scaler.fit_transform(metric_t2)
    return max_values

In [15]:
list_max_metrics = []
for metric in top_metrics:
    list_max_metrics.append(get_max_values(metric))

100% (1452 of 1452) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1452 of 1452) |####################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (1452 of 1452) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


## Creatig the graph

In [16]:
import networkx as nx
import scipy.spatial
import math
import gc

In [17]:
def create_graph():
    players_data = players_positions_FETW.copy()
    players_data = players_data.drop([0],axis=1)
    players_data.columns = [i+1 for i in range(len(players_data.columns))]
    players_data = players_data.transpose()

    def create_graph_nodes(row):
        position_list=[]
        G = nx.Graph()
        k = 0
        j = 0
        #Add the node "j", containing the argument "pos=x_coord, y_coord, for the fist team (k = 0 to 22)"
        while k < 28:
            #Apend the coordnates of the player to the postion list (will be used on a future opertion)
            #if (row[k] > -100) & (row[k+1] > -100):
            position_list.append([row[k],row[k+1]])
            G.add_node(j, pos=(row[k],row[k+1]))
            j = j+1
            k = k+2
        #Apend the graph to the graph list
        graph_list.append(G)
        j=0
        # apend the position list for the current frame to the list of positions lists (will be used on a future opertion)
        group_of_position_lists.append(position_list)

        #Repeat all for the second team"
        G = nx.Graph()
        position_list=[]
        while k < 60:
            #if (row[k] > -100) & (row[k+1] > -100):
            position_list.append([row[k],row[k+1]])
            G.add_node(j, pos=(row[k],row[k+1]))
            j = j+1
            k = k+2
        graph_list.append(G)
        #will be used to generate delaunay trianges
        group_of_position_lists.append(position_list)
        return 0

    #Transform the dataset into a numpy_array and call the function above
    x= players_data.values
    np.apply_along_axis(create_graph_nodes, 0, x)

    #create a list of tuples, containing player positions
    for a_graph in (graph_list):
        list_pos.append(nx.get_node_attributes(a_graph,'pos'))

    for pos_list in (group_of_position_lists):
        triang_del_list.append(scipy.spatial.Delaunay(pos_list))

    #create edges from delaunay triangles

    # for each set of triangles
    i = 0
    for delTri in (triang_del_list):
        edges = set()
        # for each Delaunay triangle
        for n in range(delTri.nsimplex):
        # for each edge of the triangle
        # sort the vertices
        # (sorting avoids duplicated edges being added to the set)
        # and add to the edges set
            edge = sorted([delTri.vertices[n,0], delTri.vertices[n,1]])
            edges.add((edge[0], edge[1]))
            edge = sorted([delTri.vertices[n,0], delTri.vertices[n,2]])
            edges.add((edge[0], edge[1]))
            edge = sorted([delTri.vertices[n,1], delTri.vertices[n,2]])
            edges.add((edge[0], edge[1]))
        graph = nx.Graph(list(edges)) 
        nx.set_node_attributes(graph, list_pos[i], 'pos')
        graph_group.append(graph)
        i = i+1

    # Delete edges from players with adversaries close (pos dif < 0.5) to them
    i=0
    #for i in progressbar.progressbar(range(len(list_pos))):
    while i < len(list_pos):
        for pos_id, pos_values in list_pos[i].items():
            for pos_adv, pos_values_adv in list_pos[i+1].items():
                #Get distance on the x axis
                dist_x = pos_values[0] - pos_values_adv[0]
                 #Get distance on the y axis
                dist_y = pos_values[1] - pos_values_adv[1]
                #Calculate distance
                dist_total = math.hypot(dist_y,dist_x)
                # if distance <0.5, remove the all edges by deleting and re-adding the node
                # do it for graph of both teams
                if dist_total < 0.5:
                    if graph_group[i].has_node(pos_id):
                        m_node = graph_group[i].nodes[pos_id]
                        graph_group[i].remove_node(pos_id)
                        graph_group[i].add_node(pos_id, pos=pos_values)

                    if graph_group[i+1].has_node(pos_id) & graph_group[i+1].has_node(pos_adv):
                        m_node_adv = graph_group[i+1].nodes[pos_id]
                        graph_group[i+1].remove_node(pos_adv)
                        graph_group[i+1].add_node(pos_adv, pos=pos_values_adv)
        i = i+2

In [18]:
def remove_node_graph_group(graph_number, graph_group):
    node_to_remove = []##Lista com os nós com distancia errada
    
    for node in graph_group[graph_number].nodes():
       
        try: 
            if -9999 in graph_group[graph_number].nodes[node]['pos']:
                node_to_remove.append(node)
            
        except KeyError:
             node_to_remove.append(node)
                
    for node in node_to_remove:
        graph_group[graph_number].remove_node(node)

In [19]:
#Função que pega a posição dos jogadores só dos intervalos de FETW
def get_FETW_interval(FETW_ini,FETW_end):
    list_player.append(players_positions.iloc[FETW_ini:FETW_end])

In [20]:
for partida in progressbar.progressbar(partidas):  
    
    graph_list_BPI = []#Lista da dos grafos para cada BPI
    
    #Mostra qual partida está sendo analisada
    print(partida)

    #Lendo posição dos jogadores
    players_positions = pd.read_csv('Jogos\\'+partida+'suav.2d', sep =('\s+'), header=None)

    list_player = [] #Lista com as posição de FETW
    players_positions_FETW = pd.DataFrame() #DataFrame de posição dos jogadores (considerando somente FETW)

    #Define o conjunto de dados
    game_1 = all_games[(all_games.partida == partida) & (all_games.interval_frame_size >= 500)]
    #game_1 = all_games[(all_games.partida == partida)]

    #Extraindo in
    game_1.apply(lambda x: get_FETW_interval(int(x.frame),int(x.FETW_end)),1)

    #Criando Dataframe com todas as posições de FETW
    #for jogo in list_player:
        #players_positions_FETW = pd.concat([players_positions_FETW, jogo])

    #Loop para gerar um arquivo para cada FETW, e dps com data arquivo gerar uma imagem
    FETW_count = 0
    for jogo in progressbar.progressbar(list_player):

        #Cria o dataframe dos dados do FETW atual
        players_positions_FETW = pd.DataFrame(jogo)

        #Mostra qual o FETW atual
        #print(FETW_count)

        #Define variaveis globais
        graph_list = []
        group_of_position_lists = []
        list_pos = []
        triang_del_list = []
        graph_group = []
        bet_cen_matrix_t1 = []
        bet_cen_matrix_t2 = []

        #Métos que geram os nodes
        create_graph()
        #create_digraph()
        graph_list_BPI.append(graph_group)

        #Liberando espaço
        #del triang_del_list
        del group_of_position_lists
        del list_pos
        gc.collect()

        FETW_count += 1

  0% (0 of 1) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

REDSCAT2


100% (66 of 66) |########################| Elapsed Time: 0:00:53 Time:  0:00:53
100% (1 of 1) |##########################| Elapsed Time: 0:00:56 Time:  0:00:56


## Construing the image

In [21]:
def set_one_edge(H):
    for i in H.nodes():
        H.remove_edges_from(list(H.in_edges(i))[1:])
    return H

In [39]:
def graph_image_generate(graph_group, graf_even, graf_odd, bpi_count, FETW, edges=False, 
                         node_proportion=False,title_text="",save_image=False):
    #https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
    #Lib q plota os grafos
    
    remove_node_graph_group(graf_even, graph_group) # Remove node sem posição
    remove_node_graph_group(graf_odd, graph_group)
    
    #h1 = graph_group[graf_odd].to_directed() #Transforma o grafo em direcionado
    #h2 = graph_group[graf_even].to_directed()
    
    h1 = graph_group[graf_odd]
    h2 = graph_group[graf_even]
    h3 = ball_nodes[bpi_count]
    
    #h1 = set_one_edge(h1) #Remove os edges deixando somente a primeira
    #h2 = set_one_edge(h2)
    
    my_pos=nx.get_node_attributes(h1,'pos')#Pega as posições do grafo
    my_pos_adv=nx.get_node_attributes(h2,'pos')
    ball_pos = nx.get_node_attributes(h3,'pos')
    
    img = plt.imread("soccer_field.png")#Carrega imagem de fundo

    fig, ax = plt.subplots(figsize=(12,8))#Tamanho da plotagem
    ax.imshow(img, extent=[-5,110, -5, 70])#Tamanho do campo em pixels

    #print(len(h1.nodes))
    #print(len(h2.nodes))
    
    # Gera as arestas e altera o tamanho do nó depedendo do valor da métrica
    if node_proportion & edges:
        # print('Grafo com edges')
        nx.draw_networkx(h1, my_pos, nodelist = sorted(h1.nodes),
                         node_color="#06daf9",edge_color="blue",label='Team 1',font_size=15,
                         edgelist=h1.edges, node_size=[v * 300 for v in metric_1_scaled[bpi_count] + 1])

        nx.draw_networkx(h2, my_pos_adv, nodelist = sorted(h2.nodes),
                         node_color="#d73502", edge_color="red",label='Team 2',font_size=15,
                         edgelist=h2.edges, node_size=[v * 300 for v in metric_2_scaled[bpi_count] + 1])
    
    elif edges:
        nx.draw_networkx(h1, my_pos, nodelist = sorted(h1.nodes),
                         node_color="#06daf9",edge_color="blue",label='Team 1',font_size=15,
                         edgelist=h1.edges)

        nx.draw_networkx(h2, my_pos_adv, nodelist = sorted(h2.nodes),
                         node_color="#d73502", edge_color="red",label='Team 2',font_size=15,
                         edgelist=h2.edges)
    # Caso contrário plota somente os nós
    else:
        # print('Grafo sem edges')
        nx.draw_networkx(h1, my_pos, nodelist = sorted(h1.nodes),edgelist=[],
                         node_color="#06daf9",edge_color="blue",label='Team 1',font_size=15)

        nx.draw_networkx(h2, my_pos_adv, nodelist = sorted(h2.nodes),edgelist=[],
                         node_color="#d73502", edge_color="red",label='Team 2',font_size=15)
        
    nx.draw_networkx(h3, ball_pos, node_size=100,
                     node_color="#000000", edge_color="black",label='Ball',font_size=15)

    ax.legend()
    
    if edges:
        path = 'ffmpeg_fetws/article_results_edges_eng'+str(FETW)+'/'
    else:
        path = 'ffmpeg_fetws/article_results_eng'+str(FETW)+'/'
        
    ax.title.set_text(f"Example of {title_text}")
    
    if save_image:
        Path(path).mkdir(parents=True, exist_ok=True)

        fig.savefig(path+title_text
                   +'.svg',transparent=True, format='svg')#Salva a imagem em SVG transparentere


        fig.clear()

In [23]:
def plot_generate(graph_group, graf_even, graf_odd, bpi_count):
    #https://networkx.org/documentation/stable/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
    #Lib q plota os grafos
    
    remove_node_graph_group(graf_even, graph_group) # Remove node sem posição
    remove_node_graph_group(graf_odd, graph_group)
    
    #h1 = graph_group[graf_odd].to_directed() #Transforma o grafo em direcionado
    #h2 = graph_group[graf_even].to_directed()
    
    h1 = graph_group[graf_odd]
    h2 = graph_group[graf_even]
    h3 = ball_nodes[bpi_count]
    
    #h1 = set_one_edge(h1) #Remove os edges deixando somente a primeira
    #h2 = set_one_edge(h2)
    
    my_pos=nx.get_node_attributes(h1,'pos')#Pega as posições do grafo
    my_pos_adv=nx.get_node_attributes(h2,'pos')
    ball_pos = nx.get_node_attributes(h3,'pos')
    
    img = plt.imread("soccer_field.png")#Carrega imagem de fundo

    fig, ax = plt.subplots(figsize=(12,8))#Tamanho da plotagem
    ax.imshow(img, extent=[-5,110, -5, 70])#Tamanho do campo em pixels

    #print(len(h1.nodes))
    #print(len(h2.nodes))
    
    # Gera as arestas e altera o tamanho do nó depedendo do valor da métrica

    # print('Grafo com edges')
    nx.draw_networkx(h1, my_pos, nodelist = sorted(h1.nodes),
                     node_color="#06daf9",edge_color="blue",label='Team 1',font_size=15,
                     edgelist=h1.edges)

    nx.draw_networkx(h2, my_pos_adv, nodelist = sorted(h2.nodes),
                     node_color="#d73502", edge_color="red",label='Team 2',font_size=15,
                     edgelist=h2.edges)
 
    ax.legend()  
#     ax.title.set_text("Distribuição dos jogadores em campo")
    ax.title.set_text("Distribution of players at the soccer field")

In [24]:
def generate_imagens_to_ffmpeg(bpi, edges=False):
    
    #comando para rodar e gerar um video com todas as imagens .png da pasta
    #ffmpeg -f image2 -i img%d.png test.mp4
    
    count = 0
    for graph in range(int(len(graph_list_BPI[bpi])/2)-1):
        graph_image_generate(graph_list_BPI[bpi], count, count+1, graph, bpi, edges)
        #print(count)
        count+=2
        #graph_image_generate()

In [25]:
def generate_imagens_to_ffmpeg_all_game():
    
    #comando para rodar e gerar um video com todas as imagens .png da pasta
    #ffmpeg -f image2 -i img%d.png test.mp4
    
    all_game_frame_count = 0
    for bpi in progressbar.progressbar(range(len(graph_list_BPI))):
        count = 0
        for graph in range(int(len(graph_list_BPI[bpi])/2)):
            graph_image_generate(graph_list_BPI[bpi], count, count+1, all_game_frame_count, FETW)
            #print(count)
            count+=2
            #graph_image_generate()
            all_game_frame_count+=1

## Reading the ball possession

In [26]:
ball_possition = pd.read_csv(f"ball_position/ball_position_{partidas[0]}.csv")

In [27]:
def create_ball_graph(xball):
    """
    Recives the DataFrame with ball possision and return
    a Graph with this information.
    """
    ball_nodes = []
    for bpi_ball in xball:
        G = nx.Graph()
        G.add_node('B', pos=(bpi_ball[0], bpi_ball[1]))
        ball_nodes.append(G)

    return ball_nodes

In [28]:
def get_ball_info(frames):
    """
    Return the Graph with the posistion of ball during the FETW
    (the first 167 frames on ball possession).
    """
    frame_start = frames[0]
    frame_end = frames[1]
    
    ball_posi_bpi = ball_possition.iloc[frame_start:frame_end,2:].copy()
    xball = ball_posi_bpi.values
    ball_nodes = create_ball_graph(xball)
    
    return ball_nodes

## Ploting the image

In [29]:
metric_1_scaled = list_max_metrics[0]['t1_norm']
metric_2_scaled = list_max_metrics[0]['t2_norm']

In [30]:
ball_nodes = get_ball_info(df_BPIs_info.iloc[int(list_max_metrics[0]['t1_idx'].split('_')[0])][['frame','FETW_end']])

In [31]:
graph_image_generate(graph_list_BPI[int(list_max_metrics[0]['t1_idx'].split('_')[0])],0,1,0,0,title_text='players3',save_image=True)

<Figure size 864x576 with 0 Axes>

In [32]:
top_metrics_pt = ['Eficiencia local', 'Entropia','Excentricidade']
top_metrics_eg = ['Local efficiency', 'Entropy', 'Eccentricity']

## Saving all principal metrics (both teams)

In [41]:
teams = ['t1','t2']
for team in teams:
    for principal_metric_idx in range(len(top_metrics)):
        
        #Get the number of BPI with max value of the metric
        metric_BPI_idx = split_underscore(list_max_metrics[principal_metric_idx][f'{team}_idx'],0)
        print(f'BPI utilizado {metric_BPI_idx}')

        #Get the ball localization
        ball_nodes = get_ball_info(df_BPIs_info.iloc[metric_BPI_idx][['frame','FETW_end']])

        #Normalization of the metrics values
        metric_1_scaled = list_max_metrics[principal_metric_idx]['t1_norm']
        metric_2_scaled = list_max_metrics[principal_metric_idx]['t2_norm']

        graph_image_generate(graph_list_BPI[metric_BPI_idx],
                             0,1,0,0,
                             edges=True,
                             node_proportion=True,
                             title_text=f"{top_metrics_eg[principal_metric_idx]} of {team}"
                             ,save_image=True)

BPI utilizado 58
BPI utilizado 29
BPI utilizado 16
BPI utilizado 38
BPI utilizado 37
BPI utilizado 16


<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

In [34]:
list_max_metrics

[{'t1': 0.8166666666666667,
  't1_idx': '58_locefficiency_t1.csv',
  't1_norm': array([[0.85294118, 0.66916355, 0.82528025, ..., 0.75839717, 0.75799836,
          0.7030731 ],
         [0.85294118, 0.66916355, 0.82528025, ..., 0.75839717, 0.75799836,
          0.7030731 ],
         [0.85294118, 0.66916355, 0.82528025, ..., 0.75839717, 0.75799836,
          0.7030731 ],
         ...,
         [0.15441176, 0.        , 0.04455611, ..., 0.        , 0.        ,
          0.        ],
         [0.15441176, 0.        , 0.04455611, ..., 0.        , 0.        ,
          0.        ],
         [0.15441176, 0.        , 0.04455611, ..., 0.        , 0.        ,
          0.        ]]),
  't2': 0.7999999999999999,
  't2_idx': '38_locefficiency_t2.csv',
  't2_norm': array([[0.66101695, 0.64102564, 0.61538462, ..., 0.91148491, 0.93929594,
          0.97598463],
         [0.66101695, 0.64102564, 0.61538462, ..., 0.91148491, 0.93929594,
          0.97598463],
         [0.66101695, 0.64102564, 0.61538462